In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from typing import List
from datasetsforecast.long_horizon import LongHorizon
import pandas as pd
import numpy as np
import torch
from typing import Optional

import os, pathlib
from glob import glob

from torch.utils.data import Dataset, DataLoader, Subset
from tqdm.auto import tqdm

from dataset import LongHorizonUnivariateDataModule, LongHorizonUnivariateDataset
from dataset import ElectricityUnivariateDataModule, ElectricityUnivariateDataset

import pytorch_lightning as pl
from omegaconf import OmegaConf
from utils.model_factory import instantiate

from statsforecast.models import AutoETS, ETS, Theta, AutoCES

from metrics import SMAPE, MAPE, CRPS

import os
os.environ['CUDA_VISIBLE_DEVICES']='7'

RESULTS_DIR = './results'

In [20]:
# checkpoint_pattern

In [22]:
checkpoint_name = "model-epoch=14.ckpt"

backbone = "modules.NBEATSAQFILM"
# NBEATSAQFILM NBEATSAQCAT NBEATSAQOUT "model-epoch=14.ckpt"
# AqFilmCnn AqCatCnn AqOutCnn "model-epoch=19.ckpt"
# AqCatTransformer AqOutTransformer "model-epoch=19.ckpt"
blocks = 30
lr=0.0005
width=1024
layers=3
blocks=30
warmup=400
train_q=1
quantile_embed_num=100
quantile_embed_dim=64
maxnorm=False
loss="MQNLoss"
seed="*" 

if 'NBEATS' in backbone:
    # if loss == "MQNLoss":
    #     checkpoint_pattern = f"lightning_logs/new/MHLV/model=model.AnyQuantileForecaster-backbone={backbone}-history=168-lr={lr}-width={width}-layers={layers}-blocks={blocks}-warmup={warmup}-train_q={train_q}-quantile_embed_num={quantile_embed_num}-quantile_embed_dim={quantile_embed_dim}-maxnorm={maxnorm}-loss=losses.{loss}-seed={seed}/checkpoints/{checkpoint_name}"
    # else:
    #     # checkpoint_pattern = f"lightning_logs/new/MHLV/model=model.AnyQuantileForecaster-backbone={backbone}-history=168-lr={lr}-width={width}-layers={layers}-blocks={blocks}-warmup={warmup}-train_q={train_q}-quantile_embed_num={quantile_embed_num}-quantile_embed_dim={quantile_embed_dim}-maxnorm={maxnorm}-seed={seed}/checkpoints/{checkpoint_name}"
    checkpoint_pattern = f"lightning_logs/test/MHLV/model=model.AnyQuantileForecaster-backbone={backbone}-history=168-lr={lr}-width={width}-layers={layers}-blocks={blocks}-warmup={warmup}-maxnorm={maxnorm}-loss=losses.{loss}-seed={seed}/checkpoints/{checkpoint_name}"
elif 'Cnn' in backbone:
    if loss == "MQNLoss":
        checkpoint_pattern = f"lightning_logs/test/MHLV/model=model.GeneralAnyQuantileForecaster-backbone={backbone}-history=168-lr={lr}-width={width}-train_q={train_q}-quantile_embed_num={quantile_embed_num}-quantile_embed_dim={quantile_embed_dim}-maxnorm={maxnorm}-loss=losses.{loss}-seed={seed}/checkpoints/{checkpoint_name}"
    else:
        checkpoint_pattern = f"lightning_logs/test/MHLV/model=model.GeneralAnyQuantileForecaster-backbone={backbone}-history=168-lr={lr}-width={width}-train_q={train_q}-quantile_embed_num={quantile_embed_num}-quantile_embed_dim={quantile_embed_dim}-maxnorm={maxnorm}*-seed={seed}/checkpoints/{checkpoint_name}"
elif 'Transformer' in backbone:
    # if loss == "MQNLoss":
    checkpoint_pattern = f"lightning_logs/test/MHLV/model=model.GeneralAnyQuantileForecaster-backbone={backbone}-history=168-lr={lr}-width={width}-blocks={blocks}-train_q={train_q}-quantile_embed_num={quantile_embed_num}-quantile_embed_dim={quantile_embed_dim}-maxnorm={maxnorm}-loss=losses.{loss}-seed={seed}/checkpoints/{checkpoint_name}"
    # else:
    #     checkpoint_pattern = f"lightning_logs/new/MHLV/model=model.GeneralAnyQuantileForecaster-backbone={backbone}-history=168-lr={lr}-width={width}-blocks={blocks}-train_q={train_q}-quantile_embed_num={quantile_embed_num}-quantile_embed_dim={quantile_embed_dim}-maxnorm={maxnorm}-seed={seed}/checkpoints/{checkpoint_name}"

model_list = glob(checkpoint_pattern)

cfg = torch.load(model_list[0], map_location='cpu')['hyper_parameters']
cfg = OmegaConf.create(cfg).cfg

cfg.dataset.split_boundaries = ['2006-01-01', '2017-01-01', '2018-01-01', '2019-01-01']
dm = instantiate(cfg.dataset)
dm.setup(stage='test')
test_loader = dm.test_dataloader()

model_list

['lightning_logs/test/MHLV/model=model.AnyQuantileForecaster-backbone=modules.NBEATSAQFILM-history=168-lr=0.0005-width=1024-layers=3-blocks=30-warmup=400-maxnorm=False-loss=losses.MQNLoss-seed=2/checkpoints/model-epoch=14.ckpt',
 'lightning_logs/test/MHLV/model=model.AnyQuantileForecaster-backbone=modules.NBEATSAQFILM-history=168-lr=0.0005-width=1024-layers=3-blocks=30-warmup=400-maxnorm=False-loss=losses.MQNLoss-seed=6/checkpoints/model-epoch=14.ckpt',
 'lightning_logs/test/MHLV/model=model.AnyQuantileForecaster-backbone=modules.NBEATSAQFILM-history=168-lr=0.0005-width=1024-layers=3-blocks=30-warmup=400-maxnorm=False-loss=losses.MQNLoss-seed=5/checkpoints/model-epoch=14.ckpt',
 'lightning_logs/test/MHLV/model=model.AnyQuantileForecaster-backbone=modules.NBEATSAQFILM-history=168-lr=0.0005-width=1024-layers=3-blocks=30-warmup=400-maxnorm=False-loss=losses.MQNLoss-seed=1/checkpoints/model-epoch=14.ckpt',
 'lightning_logs/test/MHLV/model=model.AnyQuantileForecaster-backbone=modules.NBEATS

In [5]:
# for ckpt in tqdm(model_list):
#     print(ckpt)
#     model = torch.load(ckpt, map_location='cpu')
#     model['hyper_parameters']['cfg']['model']['nn']['backbone']['_target_'] = 'modules.AqOutTransformer'
#     torch.save(model, ckpt)

In [23]:
dfs = []
for b in tqdm(test_loader):
    df = pd.DataFrame.from_dict({k: list(v.cpu().numpy()) for k,v in b.items() if
                                k in ['target', 'history', 'series_id', 'quantiles']})
    dfs.append(df)
df = pd.concat(dfs, axis=0, ignore_index=True)

  0%|          | 0/13 [00:00<?, ?it/s]

In [27]:
num_random = 100
num_deterministic = 101

In [25]:
def predict_checkpoint(checkpoint_file, trainer):
    cfg = torch.load(checkpoint_file)['hyper_parameters']
    cfg = OmegaConf.create(cfg).cfg
    
    model = instantiate(cfg.model, cfg).load_from_checkpoint(checkpoint_file)

    predictions = trainer.predict(model, dataloaders=dm.test_dataloader())
    predictions = torch.cat(predictions).detach().cpu()
    
    return predictions

trainer = pl.Trainer(accelerator='gpu', devices=1)
predictions = []
for ckpt in model_list:
    pred = predict_checkpoint(ckpt, trainer)
    predictions.append(pred)
    
#     predictions_deterministic = pred[...,mid_idx-num_deterministic//2:mid_idx+num_deterministic//2+1]
#     predictions_random = torch.cat([pred[..., 0:mid_idx-num_deterministic//2], 
#                                     pred[..., mid_idx+num_deterministic//2+1:]], dim=-1)
#     predictions_random, _ = torch.sort(predictions_random, dim=-1)
#     predictions_sort = torch.cat([predictions_deterministic, predictions_random], dim=-1)
    
#     predictions.append(predictions_sort)
    
predictions_ensemble = torch.stack(predictions).mean(dim=0)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [7]


Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [7]


Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [7]


...

Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [7]


Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [7]


Predicting: 0it [00:00, ?it/s]

In [28]:
crps_rnd = CRPS()
crps_fixed = CRPS()

for target, pred, q in tqdm(zip(df.target, predictions_ensemble, df.quantiles), total=len(df)):
    
    if np.isinf(target).any():
        continue

    q_deterministic = q[:num_deterministic]
    q_random = q[num_deterministic:]
    # q_random = sorted(q_random)
    
    predictions_deterministic = pred[..., :num_deterministic]
    predictions_random = pred[..., num_deterministic:]
    # predictions_random = np.sort(predictions_random, axis=-1)
    
    crps_rnd.update(preds=torch.Tensor(predictions_random)[None], 
                target=torch.Tensor(target)[None], q=torch.Tensor(q_random)[None])
    
    crps_fixed.update(preds=torch.Tensor(predictions_deterministic)[None], 
                      target=torch.Tensor(target)[None], q=torch.Tensor(q_deterministic)[None])
    
print("CRPS random quants", crps_rnd.compute().cpu().numpy())
print("CRPS mandatory quants", crps_fixed.compute().cpu().numpy())

  0%|          | 0/12740 [00:00<?, ?it/s]

CRPS random quants 211.94969
CRPS mandatory quants 209.80638


In [29]:
RESULTS_PATH = f'results/MHLV/{backbone.split(".")[-1]}-maxnorm={maxnorm}-loss={loss}'
os.makedirs(RESULTS_PATH, exist_ok=True)

for worker in tqdm(range(len(predictions))):
    p = predictions[worker]
    for series_id in df.series_id.unique():
        series_mask = df.series_id == series_id

        df_series = df[series_mask]
        p_series = p[series_mask.values].numpy()
        
        target_series = np.array([v for v in df.target[series_mask.values]])
        target_series = np.nan_to_num(target_series, posinf=np.nan)
        target_series = np.repeat(target_series[...,None], p.shape[-1], axis=-1)
        
        quantile_series = np.array([v for v in df.quantiles[series_mask.values]])
        quantile_series = np.repeat(quantile_series[:,None], p.shape[1], axis=1)
        
        forec = pd.DataFrame({f"forec{worker+1}": p_series.ravel()})
        if worker == 0:
            forec['actuals'] = target_series.ravel()
            forec['quants'] = quantile_series.ravel()
            forec = forec[['actuals', 'quants', 'forec1']]
        
        forec.to_pickle(os.path.join(RESULTS_PATH, f'e1w{worker+1}_{series_id}.pickle'))
                

  0%|          | 0/8 [00:00<?, ?it/s]